In [64]:
from sklearn import tree
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn import preprocessing,metrics 
from IPython.core.display import HTML
pd.set_option("display.max_columns",75)
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import linear_model,svm
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve

In [65]:
df = pd.read_csv(os.path.join("Loans_data_toPredict.csv"))
df.head()

,acc_now_delinq,annual_inc,application_type,avg_cur_bal,delinq_2yrs,delinq_amnt,dti,home_ownership,installment,int_rate,last_pymnt_amnt,loan_amnt,loan_status,max_bal_bc,mo_sin_rcnt_tl,mort_acc,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,open_acc,open_acc_6m,open_act_il,open_il_12m,open_il_24m,open_rv_12m,open_rv_24m,out_prncp,pct_tl_nvr_dlq,pub_rec_bankruptcies,purpose,revol_bal,revol_util,sub_grade,tax_liens,term,tot_coll_amt,tot_cur_bal,tot_hi_cred_lim,total_acc,total_bal_ex_mort,total_bal_il,total_bc_limit,total_il_high_credit_limit,total_pymnt,total_rec_late_fee,total_rev_hi_lim
0,0,55000.0,1,17223,0,0,34.50,6,391.26,11.44,391.260000,11875,Current,10023,13,2,0,3,6,4,5,14,10,12,6,14,0,0,0,14,0,3,0,2,0,2,5444.16,96.4,0,2,17823,39.4,9,0,36,0,241115,319205,28,61637,43814,24300,56205,8201.37000,0.0,45200
1,0,67000.0,1,28493,1,0,5.95,6,45.18,5.32,45.176391,1500,Current,5044,23,3,1,2,2,2,5,6,3,7,2,5,0,1,0,5,0,1,0,1,0,0,653.97,93.8,0,3,8673,43.8,1,0,36,0,142464,163700,16,10658,1985,14000,5200,947.89000,0.0,19800
2,0,114000.0,1,24630,0,0,14.04,6,1037.38,25.49,35565.100000,35000,Fully Paid,10424,0,4,4,9,11,10,11,8,14,16,11,17,0,0,1,17,1,2,0,1,1,1,0.00,87.1,0,2,35694,74.8,24,0,60,0,394086,424707,31,81780,46086,32700,52309,36875.07892,0.0,47700
3,0,56000.0,1,10710,0,0,17.90,6,370.48,6.99,8001.300000,12000,Fully Paid,1835,13,5,0,4,7,7,7,9,10,14,7,13,0,0,0,13,0,2,0,3,0,0,0.00,100.0,0,2,6665,24.0,2,0,36,0,139230,172161,28,21540,14875,22000,23500,12773.27240,0.0,27800
4,0,65000.0,1,33958,0,0,10.80,6,440.27,8.24,440.270000,14000,Current,278,0,1,0,1,2,1,1,4,2,2,2,5,0,0,1,5,1,1,0,0,1,1,6254.62,100.0,0,2,1344,20.7,6,0,36,0,169788,197948,8,20875,18203,1000,29807,9232.85000,0.0,6500


In [12]:
#Variable selection: Dropping retrospective variables that would not have been known at the time of loan issuance and thus can skew the prediction.
#df.drop(['open_acc_6m', 'open_il_12m', 'delinq_2yrs', 'num_accts_ever_120_pd', 'home_ownership', 'term', 'purpose', 'pub_rec_bankruptcies', 'num_tl_90g_dpd_24m','tax_liens', 'application_type','acc_now_delinq','delinq_amnt','num_tl_30dpd','tot_coll_amt','open_il_12m', 'open_il_24m', 'open_rv_12m','num_actv_bc_tl','mo_sin_rcnt_tl', 'num_il_tl', 'num_rev_accts','pct_tl_nvr_dlq', 'num_bc_tl','open_rv_24m', 'num_actv_rev_tl','num_op_rev_tl',
# 'num_bc_sats', 'num_sats','open_acc','num_tl_op_past_12m', 'num_rev_tl_bal_gt_0', 'mort_acc', 'open_act_il','total_rev_hi_lim', 'total_bc_limit', 'total_bal_ex_mort','total_rec_late_fee', 'sub_grade','total_acc', 'total_il_high_credit_limit', 'total_bal_il',
#'loan_amnt', 'max_bal_bc', 'revol_bal' ], axis=1, inplace=True)
#loans_df.head()


In [50]:
#Variable selection: Dropping retrospective variables that would not have been known at the time of loan issuance and thus can skew the prediction.
df.drop([ 'pub_rec_bankruptcies', 'num_tl_90g_dpd_24m','tax_liens', 'application_type','acc_now_delinq','delinq_amnt', 'num_tl_30dpd'], axis=1, inplace=True)
#loans_df.head()

In [51]:
target = df["loan_status"]
target_names = ["0", "1"]

In [52]:
data = df.drop("loan_status", axis=1)
feature_names = data.columns
data.head()

,annual_inc,avg_cur_bal,delinq_2yrs,dti,home_ownership,installment,int_rate,last_pymnt_amnt,loan_amnt,max_bal_bc,mo_sin_rcnt_tl,mort_acc,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_op_past_12m,open_acc,open_acc_6m,open_act_il,open_il_12m,open_il_24m,open_rv_12m,open_rv_24m,out_prncp,pct_tl_nvr_dlq,purpose,revol_bal,revol_util,sub_grade,term,tot_coll_amt,tot_cur_bal,tot_hi_cred_lim,total_acc,total_bal_ex_mort,total_bal_il,total_bc_limit,total_il_high_credit_limit,total_pymnt,total_rec_late_fee,total_rev_hi_lim
0,55000.0,17223,0,34.50,6,391.26,11.44,391.260000,11875,10023,13,2,0,3,6,4,5,14,10,12,6,14,0,14,0,3,0,2,0,2,5444.16,96.4,2,17823,39.4,9,36,0,241115,319205,28,61637,43814,24300,56205,8201.37000,0.0,45200
1,67000.0,28493,1,5.95,6,45.18,5.32,45.176391,1500,5044,23,3,1,2,2,2,5,6,3,7,2,5,0,5,0,1,0,1,0,0,653.97,93.8,3,8673,43.8,1,36,0,142464,163700,16,10658,1985,14000,5200,947.89000,0.0,19800
2,114000.0,24630,0,14.04,6,1037.38,25.49,35565.100000,35000,10424,0,4,4,9,11,10,11,8,14,16,11,17,1,17,1,2,0,1,1,1,0.00,87.1,2,35694,74.8,24,60,0,394086,424707,31,81780,46086,32700,52309,36875.07892,0.0,47700
3,56000.0,10710,0,17.90,6,370.48,6.99,8001.300000,12000,1835,13,5,0,4,7,7,7,9,10,14,7,13,0,13,0,2,0,3,0,0,0.00,100.0,2,6665,24.0,2,36,0,139230,172161,28,21540,14875,22000,23500,12773.27240,0.0,27800
4,65000.0,33958,0,10.80,6,440.27,8.24,440.270000,14000,278,0,1,0,1,2,1,1,4,2,2,2,5,1,5,1,1,0,0,1,1,6254.62,100.0,2,1344,20.7,6,36,0,169788,197948,8,20875,18203,1000,29807,9232.85000,0.0,6500


In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=50)

In [54]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.938935175777281

In [55]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=300)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9664702296281243

In [56]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.4156484088426042, 'out_prncp'),
 (0.2831100051541785, 'last_pymnt_amnt'),
 (0.07214524868815583, 'total_pymnt'),
 (0.0220365691042479, 'installment'),
 (0.021309321256878223, 'loan_amnt'),
 (0.010635557057875349, 'total_rec_late_fee'),
 (0.00966618080142914, 'int_rate'),
 (0.00821963844240751, 'sub_grade'),
 (0.007397457730900281, 'revol_util'),
 (0.006218666277199019, 'dti'),
 (0.005874713171977446, 'max_bal_bc'),
 (0.005866289720029831, 'annual_inc'),
 (0.005764814214133947, 'revol_bal'),
 (0.005737369706355535, 'tot_hi_cred_lim'),
 (0.005659732459828512, 'total_rev_hi_lim'),
 (0.005602614031656767, 'tot_cur_bal'),
 (0.005587441032898647, 'avg_cur_bal'),
 (0.005573127769831718, 'total_bc_limit'),
 (0.005558538710580992, 'total_bal_ex_mort'),
 (0.004882812516262143, 'total_bal_il'),
 (0.004860573569838808, 'total_acc'),
 (0.004804503545612025, 'total_il_high_credit_limit'),
 (0.004418552253715614, 'mo_sin_rcnt_tl'),
 (0.004249001255027846, 'num_il_tl'),
 (0.004129504828358582, 'nu

In [60]:
#pd.DataFrame({"feature_importances_": Feature_Importances, "feature_names": feature_names}).reset_index(drop=True)
df = pd. DataFrame(sorted(zip(rf.feature_importances_, feature_names), feature_importances: '', 'col2': [3, 4]reverse=True))
df

,0,1
0,0.415648,out_prncp
1,0.283110,last_pymnt_amnt
2,0.072145,total_pymnt
3,0.022037,installment
4,0.021309,loan_amnt
5,0.010636,total_rec_late_fee
6,0.009666,int_rate
7,0.008220,sub_grade
8,0.007397,revol_util
9,0.006219,dti


In [63]:
fig, ax = plt.subplots(figsize=(15,5))
width= .88
df = df.sort_values('0')
ax.bar(np.arange(len(feature_names)), rf.feature_importances_, width, color='b')
ax.set_xticks(np.arange(len(rf.feature_importances_)))
ax.set_xticklabels(X_train.columns.values,rotation=90)
plt.title('Features Importance')
ax.set_ylabel('Normalized Features Importance')


plt.figure(figsize=(12,8))
ax = sns.barplot(x="0", y="1", data=df, palette=sns.color_palette("GnBu", 10))
plt.xticks(rotation='vertical')
ax.grid(b=True, which='major', color='#d3d3d3', linewidth=1.0)
ax.grid(b=True, which='minor', color='#d3d3d3', linewidth=0.5)
plt.show()




ValueError: Could not interpret input '0'

<Figure size 864x576 with 0 Axes>

In [ ]:
features = ['acc_now_delinq', 'annual_inc', 'application_type', 'delinq_2yrs',
       'delinq_amnt', 'dti', 'home_ownership', 'installment', 'int_rate',
       'loan_amnt', 'max_bal_bc', 'mo_sin_rcnt_tl', 'mort_acc',
       'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl',
       'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl',
       'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_30dpd',
       'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'open_acc', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'open_rv_12m',
       'open_rv_24m', 'out_prncp', 'pct_tl_nvr_dlq', 'pub_rec_bankruptcies',
       'purpose', 'revol_bal', 'revol_util', 'sub_grade', 'tax_liens', 'term',
       'tot_coll_amt', 'tot_cur_bal', 'tot_hi_cred_lim', 'total_acc',
       'total_bal_ex_mort', 'total_bal_il', 'total_bc_limit',
       'total_il_high_credit_limit', 'total_rec_late_fee', 'total_rev_hi_lim']

In [ ]:
fig.savefig('Features_of_Importance_RF2.png')

In [ ]:

from sklearn.feature_selection import RFE
# create the RFE model and select 3 attributes
clf_LR = linear_model.LogisticRegression(C=1e30)
clf_LR.fit(X_train,y_train)
rfe = RFE(clf_LR, 10)
rfe = rfe.fit(data.iloc[:,:-1].values, data.iloc[:,-1].values)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
'open_acc_6m', 'open_il_12m', 'delinq_2yrs', 'num_accts_ever_120_pd', 'home_ownership', 'term', 'purpose', 'pub_rec_bankruptcies', 'num_tl_90g_dpd_24m','tax_liens', 'application_type','acc_now_delinq','delinq_amnt','num_tl_30dpd','out_prncp'

In [ ]:

features = ['funded_amnt','annual_inc','grade',"last_pymnt_amnt", "int_rate",
            "mo_sin_rcnt_rev_tl_op","mo_sin_old_rev_tl_op","bc_util","bc_open_to_buy","acc_open_past_24mths","loan_status"]

 'mo_sin_rcnt_tl', 'num_il_tl', 'num_rev_accts','pct_tl_nvr_dlq', 'num_bc_tl','open_rv_24m', 'num_actv_rev_tl','num_op_rev_tl',
 'num_bc_sats', 'num_sats','open_acc','num_tl_op_past_12m', 'num_rev_tl_bal_gt_0', 'mort_acc', 'open_act_il'

'total_rev_hi_lim', 'total_bc_limit', 'total_bal_ex_mort','total_rec_late_fee', 'sub_grade','total_acc', 'total_il_high_credit_limit', 'total_bal_il',
'loan_amnt'